In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch.nn as nn
import torch
import pandas as pd
import pytorch_lightning as pl
from bayesian_nnet.net_utils import initialize_pl_trainer
from bayesian_nnet.data import IQDataModel
from bayesian_nnet.cnnlstm import AutoModClassifier
from bayesian_nnet.data import initialize_data_splits
from bayesian_nnet.model_eval import init_snrid_accuracy, evaluate_model_predictions

### Initialize data split indexing

In [12]:
out_of_distrib_modes = ["lsb",
                        "usb"]

initialize_data_splits(out_of_distrib_modes)

In [13]:
iqdata = IQDataModel(batch_size=256)
model = AutoModClassifier()
callbacks, mlf_logger = initialize_pl_trainer("frequentist_net")

trainer = pl.Trainer(callbacks=callbacks,
                     max_epochs=100,
                     logger=mlf_logger)

trainer.fit(model, iqdata)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/mspcvsp/anaconda3/envs/algdev/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/mspcvsp/uncertaintyawareamc/ModelCheckpoints/frequentist_net exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                  | Params | Mode 
-----------------------------------------------------------------
0 | iq_embed       | IQDataEmbedder        | 713 K  | train
1 | modclf         | FrequentistClassifier | 526 K  | train
2 | loss           | CrossEntropyLoss      | 0      | train
3 | train_accuracy | MulticlassAccuracy    | 0      | train
4 | val_accuracy   | MulticlassAccuracy    | 0      | train
5 | test_accuracy  | MulticlassAccuracy    | 0      | train
6 | softmax        | Softmax               | 0      | train
----------------------------------------------------------------

Sanity Checking: |                                                                   | 0/? [00:00<?, ?it/s]

Training: |                                                                          | 0/? [00:00<?, ?it/s]

Validation: |                                                                        | 0/? [00:00<?, ?it/s]

Validation: |                                                                        | 0/? [00:00<?, ?it/s]

...

Validation: |                                                                        | 0/? [00:00<?, ?it/s]

Validation: |                                                                        | 0/? [00:00<?, ?it/s]

Validation: |                                                                        | 0/? [00:00<?, ?it/s]

Validation: |                                                                        | 0/? [00:00<?, ?it/s]

Validation: |                                                                        | 0/? [00:00<?, ?it/s]

In [14]:
predictions = trainer.predict(model,
                              datamodule=iqdata)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                                        | 0/? [00:00<?, ?it/s]

In [16]:
from bayesian_nnet.model_eval import ModelPredictionFormatter, evaluate_model_predictions

In [17]:
pred_fmt = ModelPredictionFormatter()
mode_conf_preds = pd.concat([pred_fmt(elem) for elem in predictions])

In [18]:
snrid_clf_report, snrid_conf_mat = evaluate_model_predictions(predictions)

ValueError: Shape of passed values is (16, 16), indices imply (18, 18)